# Transports méridiens de chaleur et de quantité de mouvement

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

Les fichiers de données au format netcdf (moyennes quotidiennes NCEP/NCAR) doivent être récupérés par FTP (FileZilla) et placés dans le répertoire data :

- Température : ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/air.XXXX.nc
- Vent zonal : ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/uwnd.XXXX.nc
- Vent méridien : ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/vwnd.XXXX.nc

Il faut télécharger au moins la période 2000-2005 et ensuite concaténer les fichier netcdf avec les commandes NCO ncrcat ou CDO mergetime :

- https://linux.die.net/man/1/ncrcat
- https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-930002.2.6

--> fichier t.nc, u.nc, v.nc

<div class="alert alert-warning">

    
<b>Transports méridiens de chaleur et de quantité de mouvement zonal par les perturbations transitoires : </b>
$$[\overline{u'v'}], [\overline{v'T'}]$$

<br>
    
<b>Transports méridiens de chaleur et de quantité de mouvement zonal par les circulations stationnaires non axisymétriques : </b>
$$[\overline{v^*}\overline{T^*}], [\overline{u^*}\overline{v^*}]$$
    
</div>

In [ ]:
%matplotlib inline

import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_zonal_mean(ax):
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 0, -100))
    ax.set_ylim(1000, 100)
    ax.set_yticks(np.arange(1000, 0, -100))  
    ax.set_xticklabels(np.arange(-90, 100, 10))
    ax.set_xticks(np.arange(-90, 100, 10)) 
    return ax

# Traitement des données

In [ ]:
year1='2000'
year2='2005'
seasons=['DJF','JJA','MAM','SON']

In [ ]:
diri="./daily_ncep/"

ft    = xr.open_dataset(diri+"t.nc").sel(time=slice(year1,year2))
fu    = xr.open_dataset(diri+"u.nc").sel(time=slice(year1,year2))
fv    = xr.open_dataset(diri+"v.nc").sel(time=slice(year1,year2))

lat  = fv.lat.values
lon  = fv.lon.values
lev = fv.level.values

print(fv)
print(lev)

plev1=1000
plev2=100
iplev1 = list(lev).index(plev1)
iplev2 = list(lev).index(plev2)

# Flux méridiens par les transitoires

$$u', v', T'$$

In [ ]:
fuprime=fu.groupby('time.dayofyear')-fu.groupby('time.dayofyear').mean('time')
uprime=fuprime['uwnd']

fvprime=fv.groupby('time.dayofyear')-fv.groupby('time.dayofyear').mean('time')
vprime=fvprime['vwnd']

ftprime=ft.groupby('time.dayofyear')-ft.groupby('time.dayofyear').mean('time')
tprime=ftprime['air']

In [ ]:
print(tprime.shape)
scale=np.cos(lat[np.newaxis,np.newaxis,:,np.newaxis]*np.pi/180.)
print(scale.shape)

$$[\overline{u'v'}], [\overline{v'T'}]$$

In [ ]:
uprime_vprime=uprime*vprime*scale  # u'v'
uprime_vprime=uprime_vprime.mean(axis=3) # moyenne zonale
uprime_vprime_an=uprime_vprime.mean(axis=0) # moyenne annuelle
uprime_vprime_season=uprime_vprime.groupby('time.season').mean('time') # moyenne saisonnière

vprime_tprime=vprime*tprime*scale  # v'T'
vprime_tprime=vprime_tprime.mean(axis=3) # moyenne zonale
vprime_tprime_an=vprime_tprime.mean(axis=0) # moyenne annuelle
vprime_tprime_season=vprime_tprime.groupby('time.season').mean('time') # moyenne saisonnière

uprime_vprime_an_vavg = uprime_vprime_an[iplev1:iplev2,:].mean(axis=0)
uprime_vprime_season_vavg = uprime_vprime_season[:,iplev1:iplev2,:].mean(axis=1)

vprime_tprime_an_vavg = vprime_tprime_an[iplev1:iplev2,:].mean(axis=0)
vprime_tprime_season_vavg = vprime_tprime_season[:,iplev1:iplev2,:].mean(axis=1)

In [ ]:
levels_vT = np.arange(-7,8,1)

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{v'T'}]$ : NCEP "+year1+"-"+year2, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vprime_tprime_an, levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vprime_tprime_an, levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/eddy_heat_flux_annual'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{v'T'}]$ : NCEP "+year1+"-"+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vprime_tprime_season[0,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vprime_tprime_season[0,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, vprime_tprime_season[1,:], levels_vT, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, vprime_tprime_season[1,:], levels_vT, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/eddy_heat_flux_seasonal'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Transport (K m/s)')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{v'T'}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : NCEP "+year1+"-"+year2)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, vprime_tprime_an_vavg, label='Annual')
plt.plot(lat, vprime_tprime_season_vavg[0,:], label='DJF')
plt.plot(lat, vprime_tprime_season_vavg[1,:], label='JJA')
plt.legend()

figname='./figs/eddy_heat_mean'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
levels_uv = np.arange(-24,26,2)

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{u'v'}]$ : NCEP "+year1+"-"+year2, fontsize=16)
ax=fig.add_subplot(111)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uprime_vprime_an, levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uprime_vprime_an, levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/eddy_momentum_flux_annual'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(12, 12))
fig.suptitle("$[\overline{u'v'}]$ : NCEP "+year1+"-"+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uprime_vprime_season[0,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uprime_vprime_season[0,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA', fontsize=14)
plot_zonal_mean(ax)
cf = ax.contourf(lat, lev, uprime_vprime_season[1,:], levels_uv, cmap='coolwarm', extend='both')
c = ax.contour(lat, lev, uprime_vprime_season[1,:], levels_uv, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('$m^2/s^2$', size='small')

figname='./figs/eddy_momentm_flux_seasonal'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('$Transport (m^2/s^2)$')
plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.title("$[\overline{u'v'}]$ - "+str(plev1)+"-"+str(plev2)+ "hPa average : NCEP "+year1+"-"+year2)
plt.axhline(0, color='black', linestyle="--")
plt.axvline(0, color='black', linestyle="--")
plt.plot(lat, uprime_vprime_an_vavg, label='Annual')
plt.plot(lat, uprime_vprime_season_vavg[0,:], label='DJF')
plt.plot(lat, uprime_vprime_season_vavg[1,:], label='JJA')
plt.legend()

figname='./figs/eddy_momentum_flux_mean'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-danger">

    
<b>En s'inspirant des codes ci-dessus, calculer et tracer les flux méridiens de chaleur et de quantité de mouvement zonal par les circulations stationnaires non axisymétriques : </b>
    
$$[\overline{v^*}\overline{T^*}], [\overline{u^*}\overline{v^*}]$$

</div>

# Flux méridiens par les circulations stationnaires non axisymétriques